### Q1. Continuing the Vector

Our Vector implementation so far looks like this

In [2]:
#example from Fluent
from array import array 
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self): 
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index): 
        cls = type(self)
        
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers' 
            raise TypeError(msg.format(cls))
            
    def __neg__(self):
        return Vector(-x for x in self) 
    
    def __pos__(self):
        return Vector(self)
    
    def _check_length_helper(self , rhs):
        if not len(self)==len(rhs):
            raise ValueError(str(self)+' and '+str(rhs)+' must have the same length')
    
    def __add__(self, rhs):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector(a + rhs for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a + b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __radd__(self, other): # other + self delegates to __add__
        return self + other

First, implement all the ad-hoc tests so far as doctests. Then implement multiplication and subtraction for this class. Notice that this is currently a non-mutable vector.

In [26]:
%%file vector_test.py
#your code here

#example from Fluent
from array import array 
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    """
    Examples
    --------
    >>> v1 = Vector([1,2])
    >>> v2 = Vector([3,4])
    >>> v3 = Vector([1,2,3])
    >>> v1+1
    Vector([2.0, 3.0])
    >>> v1+[1,2]
    Vector([2.0, 4.0])
    >>> 1 + v1
    Vector([2.0, 3.0])
    >>> [1,2] + v1
    Vector([2.0, 4.0])
    >>> v1 + v2
    Vector([4.0, 6.0])
    >>> v2 + v1
    Vector([4.0, 6.0])
    >>> v1 * v2
    Vector([3.0, 8.0])
    >>> v2 * v1
    Vector([3.0, 8.0])
    >>> 4*v1
    Vector([4.0, 8.0])
    >>> v1*4.9
    Vector([4.9, 9.8])
    >>> v1 - 0.5
    Vector([0.5, 1.5])
    >>> v1 - [10.3, -5.2]
    Vector([-9.3, 7.2])
    """
    
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self): 
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index): 
        cls = type(self)
        
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers' 
            raise TypeError(msg.format(cls))
            
    def __neg__(self):
        return Vector(-x for x in self) 
    
    def __pos__(self):
        return Vector(self)
    
    def _check_length_helper(self , rhs):
        if not len(self)==len(rhs):
            raise ValueError(str(self)+' and '+str(rhs)+' must have the same length')
    
    def _elem_op(self, rhs, op):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector( op(a, rhs) for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector( op(a, b) for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __add__(self, rhs):
        return self._elem_op(rhs, operator.add)
    
    def __radd__(self, other): # other + self delegates to __add__
        return self + other
    
    def __mul__(self, rhs):
        return self._elem_op(rhs, operator.mul)
    
    def __rmul__(self, other): # other + self delegates to __add__
        return self * other
    
    def __sub__(self, rhs):
        return self._elem_op(rhs, operator.sub)


Overwriting vector_test.py


In [28]:
!source activate py35 && py.test --doctest-modules vector_test.py

discarding /Users/victor/anaconda/bin from PATH
prepending /Users/victor/anaconda/envs/py35/bin to PATH
============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.7, py-1.4.31, pluggy-0.3.1
rootdir: /Users/victor/Documents/CS207/cs207/labs, inifile: 
plugins: cov-2.2.1
collected 1 items 

vector_test.py .

=========================== 1 passed in 0.03 seconds ===========================


In [4]:
v1 = Vector([1,2])
v2=Vector([3,4])
v3=Vector([1,2,3])
v1+1,v1+[1,2], 1 + v1, [1,2] + v1, v1 + v2, v2 + v1

(Vector([2.0, 3.0]),
 Vector([2.0, 4.0]),
 Vector([2.0, 3.0]),
 Vector([2.0, 4.0]),
 Vector([4.0, 6.0]),
 Vector([4.0, 6.0]))

In [ ]:
print(v1 * v2, v2* v1, 4*v1, v1*4.9)
print(v1 - [10.3, -5.2])

### Q2. Mixins for functionality

Here is a set of methods that logs dictionary access

In [31]:
class LoggedMappingMixin: 
    '''
    Add logging to get/set/delete operations for debugging. 
    '''
    __slots__ = ()
    def __getitem__(self, key): 
        print('Getting ' + str(key)) 
        return super().__getitem__(key)
    def __setitem__(self, key, value): 
        print('Setting {} = {!r}'.format(key, value)) 
        return super().__setitem__(key, value)
    def __delitem__(self, key): 
        print('Deleting ' + str(key)) 
        return super().__delitem__(key)

Notice the use of `super()` here. `super()` is the same as `super(self.__class__, self)`. But we dont have a parent!

What is going on? You tell me the answer to this when you inherit a `LoggedDict` with no implementation from both `LoggedMappingMixin` and `dict`. Which order must you inherit in? Play with the `mro` method and figure this out.

In [37]:
#write the LoggedDict class 2 ways and play with the mro, then write the reason
# your code here

class LoggedDict1(dict, LoggedMappingMixin):
    __foo__ = ()

class LoggedDict2(LoggedMappingMixin, dict):
    __foo__ = ()

In [49]:
#your code here

def testLogDict(c):
    print ('Trying implementation ' + c.__name__)
    a = c()
    a['foo'] = 'bar'
    del a['foo']
    print(c.mro())
    
testLogDict(LoggedDict1)
testLogDict(LoggedDict2)

Trying implementation LoggedDict1
[<class '__main__.LoggedDict1'>, <class 'dict'>, <class '__main__.LoggedMappingMixin'>, <class 'object'>]
Trying implementation LoggedDict2
Setting foo = 'bar'
Deleting foo
[<class '__main__.LoggedDict2'>, <class '__main__.LoggedMappingMixin'>, <class 'dict'>, <class 'object'>]


*your answer here*

We must inherit from LoggedMappingMixin first, and then from dict. As the MRO reveals, if we inherit from dict first, then the dictionary methods are going to come from dict and not LoggedMappingMixing. As a result, the debugging version is not used at all.

### Q3. The Pavlos Problem

ABC's and doctests. The Pavlos problem.

Introspection of a class hierarchy is helped by:
`__subclasses__()` and `_abc_registry` which give us concrete subclasses and virtual subclasses respectively. We can use this to fully document an interface via an example.

In [110]:
%%file StackInterface.py

import abc
class StackInterface(abc.ABC):
    """
    >>> a = Stack()
    >>> a.push(1)
    >>> a.push(2)
    >>> a.peek()
    2
    >>> a.pop()
    2
    >>> a.pop()
    1
    >>> a.peek()
    >>> a.pop()
    """
    
    @abc.abstractmethod
    def push(self, value):
        "Push value onto the stack. Return None"
        
    @abc.abstractmethod
    def pop(self):
        "Pop value from Stack. Return None if nothingon stack"
        
    @abc.abstractmethod
    def peek(self):
        "Peeak at top of stack. Return None if empty"

Overwriting StackInterface.py


Implement `ListStack` using a python list

In [111]:
%%file ListStack.py
#your code here
import StackInterface
import doctest

class ListStack(StackInterface.StackInterface):
    def __init__(self):
        self._stack = []
        
    def _is_empty(self):
        return not self._stack
    
    def push(self, value):
        self._stack.append(value)
    
    def pop(self):
        if not self._is_empty():
            return self._stack.pop()
        
    def peek(self):
        if self._is_empty():
            return None
        return self._stack[-1]

if __name__ == "__main__":
    a = ListStack()
    print ('Testing...')    
    doctest.run_docstring_examples(StackInterface.StackInterface, {'Stack': ListStack}, verbose=True)

Overwriting ListStack.py


How do we test this using the tests in `StackInterface`? And in general for other virtual or real subclasses? Show this here. (work out doing this from a file at home, you dont need to answer the file case here). This recipe wont work with py.test

In [112]:
#your code here
!source activate py35 && python ListStack.py

discarding /Users/victor/anaconda/bin from PATH
prepending /Users/victor/anaconda/envs/py35/bin to PATH
Testing...
Finding tests in NoName
Trying:
    a = Stack()
Expecting nothing
ok
Trying:
    a.push(1)
Expecting nothing
ok
Trying:
    a.push(2)
Expecting nothing
ok
Trying:
    a.peek()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    1
ok
Trying:
    a.peek()
Expecting nothing
ok
Trying:
    a.pop()
Expecting nothing
ok


### Q4. Your Timeseries Project

Operator overloading on your `TimeSeries` class.

Your `TimeSeries` class should be, by now, a well documented, well tested, mutable, class which implements:

- `__getitem__`: to get a value for a given time
- `__setitem__`: set the value for the given time
- `__contains__`: is a value in the values
- `__iter__`: iterates over values. (This might have iterated over tuples of (time, value) pairs earlier
- `values`: returns a numpy array of values
- `itervalues`: returns an iterator over them
- `times`: returns a numpy array of times
- `itertimes`: returns an iterator over them
- `items`: returns a list of time-value tuple pairs
- `iteritems`: returns an iterator over these
- `__len__`: returns a length.
- `__repr__`: abbreviating spring representation

Add to these methods(again well tested):

- infix addition, subtraction, equality and multiplication. Here you must check that the lengths are equal and that the time domains are the same for the case of the operations on a TimeSeries (the latter implies the former). Return a `ValueError` in case this fails:

`ValueError(str(self)+' and '+str(rhs)+' must have the same time points')`

Let these be elementwise operations, as we might expect from a numpy array-like thing. As before, handle the case of a constant.
- unary `__abs__`, `__bool__`, `__neg__`, and `__pos__` with the same semantics as the `Vector` class above.


A question that might arise is what to do if we add numpy arrays or regular python lists. These should fail with `raise NotImplemented` as we dont have time associated. An option might have been to associate the array with the time indexing of the other array, but this is making too many assumptions: the user can do this explicitly.

You will probably have to catch another exception for this to happen.

Put this code into your project repo.

### TO READ: Numpy ufuncs and function overloading

Check this out. Read http://docs.scipy.org/doc/numpy-dev/reference/arrays.classes.html#special-attributes-and-methods to understand how this works. We will use it later.

In [1]:
import numpy as np
import pandas as pd
p=pd.Series([1,2,3])
print(type(p))
p2=np.exp(p)
p2, type(p2)